### This notebook is purely for learning purpose and makes use of open sourced text-generational models to generate one-liner data using HuggingFace framework
Reference Medium Article: 

In [ ]:
# install hugging-face and all dependencies
!pip install huggingface_hub

In [ ]:
# set-up hugging face on Kaggle: https://medium.com/@kavya24goyal/setting-up-hugging-face-on-kaggle-notebooks-486e85f91cab)
# set-up hugging face on Google Colab: https://medium.com/@kavya24goyal/setting-up-hugging-face-on-google-colaboratory-49b60fc60bd0 
from huggingface_hub import notebook_login

# this steps requires your HuggingFace API Key
notebook_login()

In [ ]:
# Use a pipeline as a high-level helper and load Phi-3 model
# model documentation: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct
from transformers import pipeline
pipe = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

In [ ]:
messages = [
   {"role": "system", "content": "You are a helpful AI assistant of a software company who creates one-liner operational requests"}, 
   {"role": "user", "content": "Generate an operational request in 10-25 words"},
]


generation_args = { 
    # how many tokens you want in your output. 
    # one word is approximately 4/3 tokens
    "max_new_tokens": 50, 
    # how much randomness you want in your result OR
    # by what extent you want the model to be creative on its own
    # I am keeping it high because I do not have a source data to work on
    # and I want the data to be creative
    "temperature": 0.8, 
    "do_sample": True
}

In [ ]:
import random
import time

# clean up the response from the assistant for dataset
def sanitize_assistant_response(prompt_result):
    content = [item['content'] for item in prompt_result[0]['generated_text'] if item['role'] == 'assistant'][0]
    return content.strip().replace('"', '')

# loop and generate 1000s of requests
def ai_generated_requests(batch_size: int) -> list:
    requests = []
    start_time = time.time()
    for i in range(batch_size):
        generation_args['temperature'] = random.uniform(0.7, 1)
        prompt_result = pipe(messages, **generation_args)
        requests.append(sanitize_assistant_response(prompt_result))
        print(f'generated request {i+1}')
    return requests, time.time() - start_time


In [ ]:
# generate dataset. All your requests
requests, time_taken = ai_generated_requests(1000)
print(f'total time taken={time_taken}')